In [1]:
import os
os.chdir('/media/jorge/HDD/TFG/')
import json
import rosbag
import cv2 as cv
import numpy as np
import pyrealsense2 as rs

from cv_bridge import CvBridge
from pycocotools.coco import COCO
from skimage.feature import hog
from time import time

from src.depth2hha.getHHA import getHHA
from src.utils.gen_normal_map import depth_to_normal

BAGFILE_PATH = 'data/data_base/lindenthal-camera-traps/bagfiles'
DATASET_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/'
OUTPUT_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/images'

In [ ]:
# iterate over all the videos inside the output path
for video_folder in os.listdir(OUTPUT_PATH):
    for frame_fn in os.listdir(os.path.join(OUTPUT_PATH, video_folder, 'depth')):
        # only process the multiples of 10 plus 1
        if (int(frame_fn.split('.')[0]) - 1) % 10 != 0:
            continue

        if int(frame_fn.split('.')[0]) < 40:
            continue
        
        # read the frame
        frame = cv.imread(os.path.join(OUTPUT_PATH, video_folder, 'depth', frame_fn), cv.IMREAD_UNCHANGED)
        # equalize the histogram
        img = frame.copy()
        hist, bins = np.histogram(img.flatten(), 65536, [0, 65536])  # Collect 16 bits histogram (65536 = 2^16).
        cdf = hist.cumsum()

        cdf_m = np.ma.masked_equal(cdf, 0)  # Find the minimum histogram value (excluding 0)
        cdf_m = (cdf_m - cdf_m.min())*65535/(cdf_m.max()-cdf_m.min())
        cdf = np.ma.filled(cdf_m,0).astype('uint16')

        img2 = cdf[img]

        gradient_x = cv.Sobel(frame, cv.CV_64F, 1, 0, ksize=3)
        gradient_y = cv.Sobel(frame, cv.CV_64F, 0, 1, ksize=3)
        gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
        gradient_magnitude = cv.normalize(gradient_magnitude, None, 0, 255, cv.NORM_MINMAX)

        # Now we have the look-up table...
        

        cv.imshow('Depth', frame)
        cv.imshow('EqHist', img2)
        cv.imshow('distance_transform', gradient_magnitude)
        cv.waitKey(0)
        break


In [ ]:
# iterate over all the videos inside the output path
for video_folder in os.listdir(OUTPUT_PATH):
    for frame_fn in os.listdir(os.path.join(OUTPUT_PATH, video_folder, 'depth')):
        # only process the multiples of 10 plus 1
        # if (int(frame_fn.split('.')[0]) - 1) % 10 != 0:
        #     continue

        # if int(frame_fn.split('.')[0]) < 40:
        #     continue
        
        # read the frame
        frame = cv.imread(os.path.join(OUTPUT_PATH, video_folder, 'depth', frame_fn), cv.IMREAD_UNCHANGED)
        
        # generate the normal map from the depth map
        start = time()
        normal = depth_to_normal(frame)
        print(f'Processing time {time()-start} ms')

        cv.imshow('depth', frame)
        cv.imshow('normal', normal)
        cv.waitKey(0)
        break
        

Processing time 0.14284777641296387 ms
Processing time 0.10689640045166016 ms
Processing time 0.0989985466003418 ms
Processing time 0.09838604927062988 ms
Processing time 0.10985827445983887 ms
Processing time 0.09806656837463379 ms
Processing time 0.030026674270629883 ms
Processing time 0.029437541961669922 ms
Processing time 0.03407597541809082 ms
Processing time 0.03468132019042969 ms
Processing time 0.02533698081970215 ms
Processing time 0.0295102596282959 ms
Processing time 0.032897233963012695 ms
Processing time 0.03260207176208496 ms
Processing time 0.03409719467163086 ms
Processing time 0.030552148818969727 ms
Processing time 0.03473520278930664 ms
Processing time 0.03090190887451172 ms
Processing time 0.03336453437805176 ms
Processing time 0.034136295318603516 ms
Processing time 0.03253054618835449 ms
Processing time 0.03177475929260254 ms
Processing time 0.03307938575744629 ms
Processing time 0.03518342971801758 ms
Processing time 0.03500819206237793 ms
Processing time 0.0359

In [ ]:
# iterate over all the videos inside the output path
for video_folder in os.listdir(OUTPUT_PATH):
    for frame_fn in os.listdir(os.path.join(OUTPUT_PATH, video_folder, 'depth')):
        # only process the multiples of 10 plus 1
        # if (int(frame_fn.split('.')[0]) - 1) % 10 != 0:
        #     continue

        # if int(frame_fn.split('.')[0]) < 40:
        #     continue
        
        # read the frame
        frame = cv.imread(os.path.join(OUTPUT_PATH, video_folder, 'depth', frame_fn), cv.IMREAD_UNCHANGED)
        
        # generate the normal map from the depth map
        start = time()
        hist, bins = np.histogram(frame.flatten(), 65536, [0, 65536])  # Collect 16 bits histogram (65536 = 2^16).
        cdf = hist.cumsum()

        cdf_m = np.ma.masked_equal(cdf, 0)  # Find the minimum histogram value (excluding 0)
        cdf_m = (cdf_m - cdf_m.min())*65535/(cdf_m.max()-cdf_m.min())
        cdf = np.ma.filled(cdf_m,0).astype('uint16')
        norm_frame = cdf[frame]
        print(f'Processing time for NORM_HIST {time()-start} ms')

        # now process the hha representation
        with open(os.path.join(OUTPUT_PATH, video_folder, 'intrinsics.json'), 'r') as intrinsics_file:
            intrinsics = json.load(intrinsics_file)
        cam_matrix = np.matrix([
            [intrinsics['fx'],  0,                  intrinsics['ppx']],
            [0,                 intrinsics['fy'],   intrinsics['ppy']],
            [0,                 0,                  1]
        ])
        hha = getHHA(cam_matrix, frame/1000.0, np.zeros_like(frame))
        print(f'Processing time for HHA {time()-start} ms')

        cv.imshow('depth', frame)
        cv.imshow('normalized histogram', norm_frame)
        cv.imshow('hha', hha)
        cv.waitKey(0)
        break

Processing time for NORM_HIST 0.019161224365234375 ms
Processing time for HHA 5.2350828647613525 ms
Processing time for NORM_HIST 0.005853176116943359 ms
Processing time for HHA 4.952310800552368 ms
Processing time for NORM_HIST 0.005686521530151367 ms
Processing time for HHA 5.326498985290527 ms
Processing time for NORM_HIST 0.015009880065917969 ms
Processing time for HHA 5.369089841842651 ms
Processing time for NORM_HIST 0.008102893829345703 ms
Processing time for HHA 5.433770656585693 ms


: 

: 